# Hello, FABRIC: Create your First FABRIC Eperiment

This Jupyter notebook will walk you through creating your first FABRIC experiment. The "Hello, FABRIC" experiment tests the configuration of your environment to ensure you can create and access resources on FABRIC.  Specifically, the experiment deploys a slice of resources with a single virtual machine and confirms you can log into the machine. 

## Step 1:  Configure the Environment

Set the environment variables that will be used by this notebook. If you are using the FABRIC JupyterHub many of the environment varialbe will be automatically configured for you.  You will only need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  If you do not have a bastion username and private key, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

## Step 2: Import the FABLlib Library

In [ ]:
import json
import traceback
#from fabrictestbed_extensions.fablib.fablib import fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager


#fablib.show_config()
#fablib = fablib()

#fablib = fablib_manager()

fablib = fablib_manager(
                     #fabric_rc='/Users/pruth/other_fabric_rc',
                     #credmgr_host='credmgr_host_xxx',
                     #orchestrator_host='orchestrator_host_xxx',
                     #fabric_token='token_xxx',
                     #project_id='id_xxx',
                     #bastion_username='bastion_name_xxx',
                     #bastion_key_filename='bastion_key_xxx',
                     #log_level='DEBUG',
                     #log_file='myfile'
                     )
print("\n\n")
fablib.show_config()

## Step 3 (Optional): Query for Available Tesbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Step 4: Create the Experiment Slice

The following creates a single node with basic compute capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

By default, the submit function will block until the node is ready and will display the progress of your slice being built.


<img src="./figs/SingleNode.png" width="20%"><br>



In [ ]:
try:
    

    #Create a slice
    slice = fablib.new_slice(name="MySlice3305")
    
    # Add a node
    #site='UTAH'
    #host_name=f'{site.lower()}-w1.fabric-testbed.net'
    #node = slice.add_node(name=f"Node1_{site}", site=site)
    #iface1 = node.add_component(model='NIC_ConnectX_5', name=f"nic1_{site}").get_interfaces()[0]
    
    site='UTAH'
    node = slice.add_node(name=f"Node1_{site}", site=site, cores=2, ram=8, disk=10, image='default_ubuntu_20')
    iface2 = node.add_component(model='NIC_ConnectX_5', name=f"nic1_{site}").get_interfaces()[0]
    print(f"iface2.get_vlan(): {iface2.get_vlan()}")

    #iface2.set_vlan(3305)
    

    facility_port = slice.add_facility_port(name='Chameleon-StarLight', site='STAR', vlan='3305')
    facility_port_interface =facility_port.get_interfaces()[0]
    print(f"facility_port_interface.get_vlan(): {facility_port_interface.get_vlan()}")

    #net = slice.add_l2network(name=f'net_facility_port', interfaces=[iface2,facility_port_interface], type='L2Bridge')
    net = slice.add_l2network(name=f'net_facility_port', interfaces=[iface2,facility_port_interface], type='L2PTP')
    
    print(f"AFTER: facility_port_interface.get_vlan(): {facility_port_interface.get_vlan()}")
    print(f"AFTER: iface2.get_vlan(): {iface2.get_vlan()}")
        

    #Submit the Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Step 5: Observe the Slice's Attributes

### Print the slice 

In [ ]:
slice_name='MySlice3305'

try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

### Print the node

Each node in the slice has a set of get functions that return the node's attributes.

In [ ]:
try:
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

## Step 6: Run the Experiemnt

Most experiments will require automated configuration and execution. You can use the fablib library to execute arbitrary commands on your node. 

The following code demonstrates how to use fablib to execute a "Hello, FABRIC" bash script. The library uses the bastion and VM keys defined at the top of this notebook to jump through the bastion host and execute the script.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("192.168.42.0/24")
    available_ips = list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    node1 = slice.get_node(name=f"Node1_{site}")        
    node1_iface = node1.get_interface(network_name=f'net_facility_port') 
    node1_addr = available_ips.pop(99)
    print(f"node1_addr: {node1_addr}")
    node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)

    stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_physical_os_interface_name()} up')
    print (stdout)
    
    stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_os_interface()} up')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    node = slice.get_node(name=f"Node1_{site}")        
    stdout, stderr = node.execute('echo Hello, FABRIC from node `hostname -s`')
    print(stdout)
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    node1 = slice.get_node(name=f"Node1_{site}")     
    node1_iface = node1.get_interface(network_name=f'net_facility_port') 


    stdout, stderr = node1.execute(f'ping -c 5 192.168.42.9')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

## Step 7: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")